# CIFAR10 - Use Dilated & Depthwise Separable Convolutions

### The requirements in detail:


- change the code such that it uses GPU
- change the architecture to C1C2C3C40 (basically 3 MPs)
- total RF must be more than 44
- one of the layers must use Depthwise Separable Convolution
- one of the layers must use Dilated Convolution
- use GAP (compulsory):- add FC after GAP to target #of classes (optional)
- achieve 80% accuracy, as many epochs as you want. Total Params to be less than 1M. 
- upload to Github
- Attempt S7-Assignment Solution


## Setup google drive for storage & retrieval

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

module_folder = 'drive/My Drive/eva_stored_from_colab/eva5/cifar10_convs/'
base_folder = module_folder+'trained_weights/'
anlt_folder = module_folder+'analysis_materials/'
acc_recorder_files = ["maxAcc_cifar10_convs"]
model_file_names = ["mdl_cifar10_convs"]

Mounted at /content/drive


## Get module for github & set it up for use

In [10]:
# import os
# from pathlib import Path
# import shutil
# import time

# raw_models_dir = Path(module_folder + "raw_models/")
# custom_utils_dir = Path(module_folder + "custom_utils/")

# if raw_models_dir.exists():
#     shutil.rmtree(raw_models_dir)

# if custom_utils_dir.exists():
#     shutil.rmtree(custom_utils_dir)

# time.sleep(60)

# !git clone  https://github.com/sairamsubramaniam/tsai_projects.git
# !mv 'tsai_projects/assignment7_cifar10/raw_models' 'drive/My Drive/eva_stored_from_colab/eva5/cifar10_convs/'
# !mv 'tsai_projects/assignment7_cifar10/custom_utils' 'drive/My Drive/eva_stored_from_colab/eva5/cifar10_convs/'

# try:
#     os.mkdir('drive/My Drive/eva_stored_from_colab/eva5/cifar10_convs/trained_weights')
# except:
#     pass

# !rm -r  'tsai_projects/'

Cloning into 'tsai_projects'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 106 (delta 58), reused 84 (delta 39), pack-reused 0
Receiving objects: 100% (106/106), 601.85 KiB | 21.49 MiB/s, done.
Resolving deltas: 100% (58/58), done.


## Import all the required modules including the custom ones

In [2]:
from __future__ import print_function
from operator import itemgetter
import time
import sys
sys.path.append(module_folder)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
%matplotlib inline

#import raw_models
from raw_models import S7
import custom_utils
from custom_utils import helpers
from custom_utils import analytics

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

## Download Data

In [3]:
torch.manual_seed(1)
batch_size = 128

transform = transforms.Compose(
                          [transforms.RandomRotation(10) ,
                           transforms.RandomHorizontalFlip(0.25) ,
                           transforms.ToTensor(),
                          transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## Check the number of parameters

In [4]:
!pip install torchsummary
from torchsummary import summary
summary(S7.S7().to(device), input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             448
       BatchNorm2d-2           [-1, 16, 32, 32]              32
              ReLU-3           [-1, 16, 32, 32]               0
       BatchNorm2d-4           [-1, 16, 32, 32]              32
              ReLU-5           [-1, 16, 32, 32]               0
            Conv2d-6           [-1, 32, 32, 32]           4,640
       BatchNorm2d-7           [-1, 32, 32, 32]              64
              ReLU-8           [-1, 32, 32, 32]               0
       BatchNorm2d-9           [-1, 32, 32, 32]              64
             ReLU-10           [-1, 32, 32, 32]               0
        MaxPool2d-11           [-1, 32, 16, 16]               0
           Conv2d-12           [-1, 32, 16, 16]           1,056
      BatchNorm2d-13           [-1, 32, 16, 16]              64
             ReLU-14           [-1, 32,

## Train - 80% achieved by 8th Epoch

In [5]:
fn_idx = 0
epochs = 26

model = S7.S7().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=0, momentum=0.9)
loss_func = F.nll_loss

acc_recorder_file = acc_recorder_files[fn_idx]
model_file_name = model_file_names[fn_idx]

accuracy_store_path = base_folder+acc_recorder_file
model_sd_save_path = base_folder+model_file_name+"_sd.pth"


trial1 = helpers.train_epochs(
                    model=model, 
                    device=device, 
                    train_loader=train_loader,
                    test_loader=test_loader, 
                    optimizer=optimizer, 
                    loss_func=loss_func, 
                    epochs=epochs,
                    accuracy_store_path=accuracy_store_path, 
                    model_sd_save_path=model_sd_save_path,
                    save_if_better_acc=True)

  0%|          | 0/391 [00:00<?, ?it/s]


LAST RECORDED MAX ACCURACY:  0.0

EPOCH NUM 1


loss=1.446407437324524 batch_id=390: 100%|██████████| 391/391 [00:19<00:00, 20.52it/s]



Train Data: Average loss: 0.0117, Accuracy: 22504/50000 (45.0080%)



  0%|          | 0/391 [00:00<?, ?it/s]

...


  0%|          | 0/391 [00:00<?, ?it/s]


Test set: Average loss: 0.5224, Accuracy: 8416/10000 (84.1600%)

-----------------------------------------------

EPOCH NUM 25


loss=0.2167029082775116 batch_id=390: 100%|██████████| 391/391 [00:17<00:00, 21.81it/s]


Train Data: Average loss: 0.0017, Accuracy: 46239/50000 (92.4780%)




Test set: Average loss: 0.5578, Accuracy: 8319/10000 (83.1900%)

-----------------------------------------------
TOTAL TRAINING TIME:  528.2712850570679
LAST 10 EPOCH AVG ACC:  0.8309899999999999
LAST 5 EPOCH AVG ACC:  0.83614
MAX ACCURACY:  0.8416
